# Future를 이용한 동시성

이 장에서는 PyPI의 futures 패키지를 통해 사용할 수 있는 concurrent.futures 라이브러리를 중점적으로 알아본다.

그리고 비동기 작업의 실행을 나타내는 객체인 Future의 개념에 대해 소개한다. 이 강력한 개념은 concurrent.futures뿐만 아니라 asyncio 패키지의 기반이 된다.

## 예제: 세 가지 스타일의 웹 내려받기

긴 지연시간 동안 CPU 클록을 낭비하지 않기 위해 네트워크 입출력을 효율적으로 처리하려면 동시성을 이용해야 한다. 네트워크 응답이 오는 동안 다른 일을 처리하는 것이 좋다.

In [9]:
# 순차 내려받기 스크립트

import os
import time
import sys

import requests

POP20_CC = ('CN IN US ID BR PK NG BD RU JP '
            'MX PH VN ET EG DE IR TR CD FR').split()

BASE_URL = 'http://flupy.org/data/flags'
DEST_DIR = os.getcwd() + '/downloads/'

def save_flag(img, filename):
  path = os.path.join(DEST_DIR, filename)
  with open(path, 'wb') as fp:
    fp.write(img)

def get_flag(cc):
  url = '{}/{cc}/{cc}.gif'.format(BASE_URL, cc=cc.lower())
  resp = requests.get(url)
  return resp.content

def show(text):
  print(text, end=' ')
  sys.stdout.flush()

def download_many(cc_list):
  for cc in sorted(cc_list):
    image = get_flag(cc)
    show(cc)
    save_flag(image, cc.lower() + '.gif')
  return len(cc_list)

def main(download_many):
  t0 = time.time()
  count = download_many(POP20_CC)
  elapsed = time.time() - t0
  msg = '\n{} flags downloaded in {:.2f}s'
  print(msg.format(count, elapsed))

if __name__ == '__main__':
  main(download_many)


BD BR CD CN DE EG ET FR ID IN IR JP MX NG PH PK RU TR US VN 
20 flags downloaded in 18.45s


## concurrent.futures로 내려받기

`concurrent.futures` 패키지의 가장 큰 특징은 ThreadPoolExecutor와 ProcessPoolExecutor 클래스인데, 이 클래스들은 콜러블 객체를 서로 다른 스레드나 프로세스에서 실행할 수 있게 해주는 인터페이스를 구현한다.
이 클래스들은 작업자 스레드나 프로세스를 관리하는 풀과 실행할 작업을 담은 큐를 가지고 있다. 그러나 아주 고수준의 인터페이스를 구현하고 있어서 국기를 내려받는 간단한 프로그램을 구현할 때는 내부의 작동과정을 알 필요가 없다.

In [11]:
from concurrent import futures

MAX_WORKERS = 20

def download_one(cc):
  image = get_flag(cc)
  show(cc)
  save_flag(image, cc.lower() + '.gif')
  return cc

def download_many(cc_list):
  workers = min(MAX_WORKERS, len(cc_list))
  with futures.ThreadPoolExecutor(workers) as executor:
    res = executor.map(download_one, sorted(cc_list))

  return len(list(res))

if __name__ == '__main__':
  main(download_many)

TR BD EG IR DE ID FR BR PK PH RU CN MX CD US NG JP VN IN ET 
20 flags downloaded in 1.54s
